In [ ]:
# install dependencies if not exists
!pip3 install tensorflow==2.3

In [22]:
# import modules
import tensorflow
from tensorflow import keras 
import logger 
import os
import time

logger.debug('Tensorflow version', tensorflow.__version__)

[DEBUG, "/usr/lib/python3.8/runpy.py":194, in _run_module_as_main] Tensorflow version 2.3.0


In [65]:
class FaceWrapper:
    def __init__(self):
        # configs goes here
        self.faceWidth = 128
        self.faceHeight = 128
        self.batchSize = 32
        self.datasetDir = './faces'

    def importDataset(self):
        # preprocess training data
        train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=1./3)

        # split dataset
        train_set = train_datagen.flow_from_directory(
            self.datasetDir,
            seed=10,
            target_size=(self.faceHeight, self.faceWidth),
            class_mode='categorical', 
            batch_size=self.batchSize, 
            subset="training",
        )

        test_set = train_datagen.flow_from_directory(
            self.datasetDir,
            seed=10,
            target_size=(self.faceHeight, self.faceWidth),
            class_mode='categorical', 
            batch_size=self.batchSize, 
            subset="validation"
        )

        return train_set, test_set

    def buildModel(self):
        model = keras.Sequential([
            keras.layers.Flatten(input_shape=(self.faceHeight, self.faceWidth, 3)),
            keras.layers.Dense(32, activation='relu'),
            keras.layers.Dropout(0.2),
            keras.layers.Dense(2, activation='softmax'),
        ])
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    
    def runWorkFlow(self, epochs=10): # exec the whole workflow
        train, test = self.importDataset()
        self.model = self.buildModel()
        logger.debug(train, test)
        self.train(train, test, epochs=epochs)
        
    def train(self, train_set, test_set, epochs=10):
        self.model.fit(
            train_set, # data to train, a format of (X, y)
            steps_per_epoch=len(train_set), # The number of batch iterations before a training epoch is considered finished. Ignore if whole data is load, in our case, we need this in order to iterate over batches; hence make sure that generator can generate at least: steps_per_epoch * epochs batches
            epochs=epochs, # number of epochs, i.e times that iteration of updating weights goes
            validation_data=test_set, # validation
            validation_steps=len(test_set)
        )

    def predict(self, X):
        return self.model.predict(X)
    def save(self, id):
        self.model.save(id)
    def load(self, id):
        self.model = keras.models.load_model(id)

In [69]:
face = FaceWrapper()

face.runWorkFlow()

Found 450 images belonging to 2 classes.
Found 224 images belonging to 2 classes.
[DEBUG, "/usr/lib/python3.8/runpy.py":194, in _run_module_as_main] <tensorflow.python.keras.preprocessing.image.DirectoryIterator object at 0x7f128445c280> <tensorflow.python.keras.preprocessing.image.DirectoryIterator object at 0x7f1278625070>
Epoch 1/10
15/15 [==============================] - 1s 36ms/step - loss: 2.5832 - accuracy: 0.6778 - val_loss: 1.3256 - val_accuracy: 0.4955
Epoch 2/10
15/15 [==============================] - 0s 32ms/step - loss: 0.3328 - accuracy: 0.8800 - val_loss: 0.2841 - val_accuracy: 0.8795
Epoch 3/10
15/15 [==============================] - 0s 31ms/step - loss: 0.1900 - accuracy: 0.9467 - val_loss: 0.1819 - val_accuracy: 0.9777
Epoch 4/10
15/15 [==============================] - 0s 32ms/step - loss: 0.1499 - accuracy: 0.9511 - val_loss: 0.1895 - val_accuracy: 0.9286
Epoch 5/10
15/15 [==============================] - 0s 30ms/step - loss: 0.1567 - accuracy: 0.9600 - val_loss

In [68]:
# load model
face = FaceWrapper() 
face.load('fc_2cls.h5')
print(face.model.summary())

# save model
face.save('fc_saved.h5')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 49152)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1572896   
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
Total params: 1,572,962
Trainable params: 1,572,962
Non-trainable params: 0
_________________________________________________________________
None
